# Steady state and Step response

Thermal Model Simulation.
This notebook simulates a simplified thermal model of a building using the `dm4bem` library in Python. It compares indoor temperature evolution using both explicit and implicit Euler methods, with and without a controller.

Imports and Configuration Load  Python libraries like numpy, pandas, matplotlib, and dm4bem

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import dm4bem


Model Setup we neglected the glass capacity because we took the equivalent value which is combined together.

In [ ]:
controller = False
neglect_air_glass_capacity = True 
imposed_time_step = True
Δt = 1800  # s,imposed time step


Load Thermal Circuit from CSV

The thermal circuit is loaded using the 'dm4bem.file2TC()' function, which reads a '.csv' file containing the thermal model elements.

If the 'controller' flag is set to 'True', we simulate an active control system by assigning a high thermal conductance value (1000 W/K) to nodes 'q6' and 'q14'. This represents strong temperature regulation — similar to a thermostat — which ensures the room temperature closely follows the set point.


In [ ]:
TC = dm4bem.file2TC('./MODEL/TC.csv', name='', auto_number=False)

if controller:
    TC['G']['q6'] = 1e3
    TC['G']['q14'] = 1e3


now Convert to State-Space and Determine Time Step

The thermal circuit model is converted to its state-space representationusing the 'dm4bem.tc2ss()' function, resulting in matrices 'A', 'B', 'c', and 'D', which describe the system's dynamics.

The eigenvalues of matrix 'A' are used to calculate the maximum stable time step('Δtmax') for the explicit Euler method. This ensures numerical stability.


In [ ]:
[As, Bs, Cs, Ds, us] = dm4bem.tc2ss(TC)
λ = np.linalg.eig(As)[0]

Δtmax = 2 * min(-1 / λ)
dt = Δt if imposed_time_step else dm4bem.round_time(Δtmax)

t_settle = 4 * max(-1 / λ)
duration = np.ceil(t_settle / 3600) * 3600


Now Generates input and time Vector

In this step, we create the input signals for the thermal model over the simulation period.

'To': Outdoor temperature is set to a constant 10 °C.
'Ti1', 'Ti2': Indoor set point temperatures are both set to 20 °C.
'Φi1', 'Φi2', 'Φo1', 'Φo2': All solar radiation and auxiliary heating inputs are set to 0 (no heat gains/losses from solar or internal sources).

These arrays are then combined into a DataFrame 'input_data_set', with the simulation 'time' as the index. This will be used to feed the model with time-varying (or constant) boundary conditions.


In [ ]:
To = 10 * np.ones(n)         # outdoor temperature 10 °C
Ti1 = 20 * np.ones(n)        # indoor temperature set point 20 °C
Ti2 = 20 * np.ones(n)        # indoor temperature set point 20 °C
Φi1 = 0 * np.ones(n)         # solar radiation absorbed by the glass
Φo2 = Φo1 = Φi2 = Φi1        # auxiliary heat sources and solar radiation


data = {'To': To, 'Ti1': Ti1, 'To': To, 'Ti2': Ti2, 'To': To, 'Φo1': Φo1, 'Φi1': Φi1, 'Φi2': Φi2, 'Φo2': Φo2}
input_data_set = pd.DataFrame(data, index=time)

Time Integration (Euler) Computes indoor temperatures using explicit and implicit Euler methods.

In this step, we simulate how indoor temperatures evolve over time using two numerical methods:

-Explicit Euler: Calculates the next state directly using the current state.
-Implicit Euler: Uses the next state in the equation (requires solving a matrix inverse at each step).

We initialize both temperature dataframes ;θ_exp' and 'θ_imp' with zeros as the initial indoor temperature. Then, using a loop, we compute temperatures at each time step using the state-space matrices 'As' and 'Bs'.

The identity matrix 'I' is used to match the size of matrix 'As' for numerical operations.


In [ ]:
θ_exp = pd.DataFrame(index=u.index)
θ_imp = pd.DataFrame(index=u.index)

θ0 = 0.0
θ_exp[As.columns] = θ0
θ_imp[As.columns] = θ0

I = np.eye(As.shape[0])
for k in range(u.shape[0] - 1):
    θ_exp.iloc[k + 1] = (I + dt * As) @ θ_exp.iloc[k] + dt * Bs @ u.iloc[k]
    θ_imp.iloc[k + 1] = np.linalg.inv(I - dt * As) @ (θ_imp.iloc[k] + dt * Bs @ u.iloc[k])


Results and plotting

This section visualizes the indoor temperatures computed by both explicit and implicit Euler methods.

-The output temperatures y_exp and y_imp are calculated using the state-space output equation:  
 nated into a single DataFrame y for easy plotting.

The graph displays the temperature evolution over time for Room 1 and Room 2, allowing comparison between both numerical methods. The plot title also shows the chosen and maximum allowable time step.


In [ ]:
y_exp = (Cs @ θ_exp.T + Ds @ u.T).T
y_imp = (Cs @ θ_imp.T + Ds @ u.T).T

y = pd.concat([y_exp, y_imp], axis=1)
y.columns = ['y_explicit_Room1', 'y_explicit_Room2', 'y_implicit_Room1', 'y_implicit_Room2']

ax = y.plot()
ax.set_xlabel('Time')
ax.set_ylabel('Indoor temperature, $\\theta_i$ / °C')
ax.set_title(f'Time step: dt = {dt:.0f} s; dt_max = {Δtmax:.0f} s')
plt.grid()
plt.show()


 FinallySteady-State Temperature Output

In [ ]:
print("Steady-state indoor temperature:")
print(f"- Room 1: {y_exp.iloc[-1, 0]:.2f} °C")
print(f"- Room 2: {y_exp.iloc[-1, 1]:.2f} °C")
